In [1]:
import pandas as pd
import numpy as np
import branca
import folium
import csv , json, os
import geocoder
import ipywidgets

from folium.features import CustomIcon
from PIL import Image
from folium.plugins import MiniMap
from folium import plugins
from folium.plugins import Search
from folium.plugins import MarkerCluster
from branca.colormap import linear


with open('ncr_city.geojson', 'r') as json_file:
    ncr_city_geojson = json.load(json_file)

with open('Barangay.json', 'r') as json_file:
    barangay = json.load(json_file)

df = pd.read_csv('ncrbra_2.csv')
#df.drop('Unnamed: 0', axis=1, inplace=True)
df.dropna(inplace=True)
ncr_city = pd.read_csv('ncr_city.csv')
lst = ['latitude','longitude','altitude']
ncr_city.drop(lst, axis=1, inplace=True)

df.Confirmed_Cases = df.Confirmed_Cases.astype(int)
df.head()

,Barangay,City,Latitude,Longitude,Population,Confirmed_Cases
0,Acacia,Malabon,14.6691,120.9658,5127.0,1
1,Addition Hills,Mandaluyong,14.5862,121.0374,99508.0,7
2,Addition Hills,San Juan,14.5938,121.0385,3739.0,4
3,Aguho,Pateros,14.5440,121.0632,65322.0,0
4,Alabang,Muntilupa,14.4187,121.0405,63793.0,16


In [2]:
nc = df.sort_values('City', ascending=False)
nc.to_csv('NCR_BARANGAY.csv')

In [3]:
qc = df[df.City == 'Quezon City']
qc.to_csv('QuezonCity.csv')
pasay = df[df.City =='Pasay']
pasay.to_csv('pasay.csv')

In [4]:
colormap = linear.Reds_09.scale(
    ncr_city.ConfirmedCases.min(),
    ncr_city.ConfirmedCases.max())
ncr_dict = ncr_city.set_index('City')['ConfirmedCases']

In [5]:
legend_html = '''
{% macro html(this, kwargs) %}
<div style="
    position: fixed; 
    bottom: 50px;
    left: 50px;
    width: 280px;
    height: 80px;
    z-index:9999;
    font-size:14px;
    ">
    <p><a style="color:#e6194b;font-size:150%;margin-left:20px;">◼</a>&emsp;Confirmed Case/s</p>
    <p><a style="color:#19e6b4;font-size:150%;margin-left:20px;">◼</a>&emsp;No Confirmed Case/s</p>
</div>
<div style="
    position: fixed; 
    bottom: 50px;
    left: 50px;
    width: 200px;
    height: 80px; 
    z-index:9998;
    font-size:14px;
    background-color: #ffffff;
    filter: blur(8px);
    -webkit-filter: blur(8px);
    opacity: 0.7;
    ">
</div>
{% endmacro %}
'''

legend = branca.element.MacroElement()
legend._template = branca.element.Template(legend_html)

In [9]:
m = folium.Map(location=(14.6091, 121.0223), zoom_start=11.5)

marker_cluster = MarkerCluster(name='Confirmed Cases').add_to(m).add_child(legend)
marker_cluster_1 = MarkerCluster(name='No Confirmed Cases').add_to(m).add_child(legend)






for lat, lot, brgy, cases in zip(df['Latitude'],df['Longitude'],df['Barangay'],df['Confirmed_Cases']):
    if cases > 0:
        folium.Marker(
        location=[lat,lot],
        popup=('<b>Barangay:</b><span style=\"color: blue;\"> ' + str(brgy) + '</span><br>'
         '<b>Confirmed Cases:<b/><span style=\"color: red\"> ' + str(cases) + '</span>'),
        icon=folium.Icon(color='red', icon='fa-user-times', prefix='fa'),
        name='1-2 Confirmed Cases',
        icon_function="""
        function(cluster){
        return L.divIcon({html: '<b>' + cluster.getChildCount() + '</b>',
        className: 'marker-cluster marker-cluster-small',
         iconSize: new L.Point(20, 20)});
        }""").add_to(marker_cluster)
    else:
        folium.Marker(
        location=[lat,lot],
        popup=('<b>Barangay:</b><span style=\"color: blue;\"> ' + str(brgy) + '</span><br>'
         '<b>Confirmed Cases:<b/><span style=\"color: green\"> ' + str(cases) + '</span>'),
        icon=folium.Icon(color='green', icon='fa-users', prefix='fa'),
        name='No Confirmed Cases'
        ).add_to(marker_cluster_1)

city_geo = folium.GeoJson(
    ncr_city_geojson,
    name='City',
   # show=False,
    #0.7
    tooltip=folium.GeoJsonTooltip(fields=['City','ConfirmedCases'],
                                  aliases=['City: ','Confirmed Cases:'],
                                  localize=True,
                                    labels=True),
    highlight_function=lambda x: {'fillOpacity':0.1},
    style_function=lambda feature: {
        'fillColor':colormap(ncr_dict[feature['properties']['City']]),
        'fillOpacity': 0.7,
        'color': 'black',
        'weight': 3.0}).add_to(m)  
    
citysearch = Search(
    layer=city_geo,
    geom_type='Polygon',
    placeholder='Search for  City',
    collapsed=True,
    search_label='City',
    weight=3
).add_to(m)



colormap.caption = 'Confirmed Cases of Covid-19'
colormap.add_to(m)

plugins.Fullscreen(
position='topright',
title='Expand me',
title_cancel='Exit me',
force_separate_button=True).add_to(m)

folium.LayerControl(collapsed=True).add_to(m)


#minimap = MiniMap(toggle_display=False)


m


In [85]:
m.save('index.html')

In [14]:
df.isnull().sum()

Barangay           1
City               1
Latitude           1
Longitude          1
Population         1
Confirmed_Cases    1
dtype: int64

m = folium.Map(location=(14.6091, 121.0223), zoom_start=11.5)

marker_cluster = MarkerCluster(name='Marker').add_to(m).add_child(legend)





for lat, lot, brgy, cases in zip(df['Latitude'],df['Longitude'],df['Barangay'],df['Confirmed_Cases']):
    if cases > 0:
        folium.Marker(
        location=[lat,lot],
        popup=('<b>Barangay:</b><span style=\"color: blue;\"> ' + str(brgy) + '</span><br>'
         '<b>Confirmed Cases:<b/><span style=\"color: red\"> ' + str(cases) + '</span>'),
        icon=folium.Icon(color='red', icon='remove-sign'),
        name='1-2 Confirmed Cases'
        ).add_to(marker_cluster)
    else:
        folium.Marker(
        location=[lat,lot],
        popup=('<b>Barangay:</b><span style=\"color: blue;\"> ' + str(brgy) + '</span><br>'
         '<b>Confirmed Cases:<b/><span style=\"color: green\"> ' + str(cases) + '</span>'),
        icon=folium.Icon(color='green', icon='ok sign'),
        name='No Confirmed Cases'
        ).add_to(marker_cluster)

folium.GeoJson(
    ncr_city_geojson,
    name='City',
    #0.7
    tooltip=folium.GeoJsonTooltip(fields=['City','ConfirmedCases'],
                                  aliases=['City: ','Confimred Cases:']),
    highlight_function=lambda x: {'fillOpacity':1.0},
    style_function=lambda feature: {
        'fillColor':colormap(ncr_dict[feature['properties']['City']]),
        'color': 'black',
        'weight': 0.5}).add_to(m)  
    
    

plugins.Fullscreen(
position='topright',
title='Expand me',
title_cancel='Exit me',
force_separate_button=True).add_to(m)
colormap.caption = 'Confirmed Cases of Covid-19'
colormap.add_to(m)
folium.LayerControl().add_to(m)

minimap = MiniMap(toggle_display=True)
m.add_child(minimap)
m.save('NCR.html')
m
